In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
def pointsAboveReplacement(file):
    with open(file, "r") as infile:
        data = json.load(infile)
    for i, player in enumerate(data):
        if i < (len(data)-1):
            data[i].update({"points_above_replacement": player["togga_score"] - data[i+1]["togga_score"]})   
    with open(file, 'w') as json_data:
        json.dump(data, json_data)

for position in ['goalies', 'defense', 'midfield', 'forwards']:
    for year in ['2012/13', '2013/14', '2014/15', '2015/16', '2016/17']:
        file = 'Data/Sorted_Positions/sorted_%s_%s+%s.json' % (position, year[:4], year[-2:])
        pointsAboveReplacement(file)



In [172]:
with open(file, "r") as infile:
        data = json.load(infile)

points_above = np.array([(x["name"], x["points_above_replacement"]) for x in data[:-1]])
#print(points_above)


In [95]:
# Removed ontarget_scoring_att
relevant_forward_keys = ['togga_score', 'apps', 'tackle_total', 'yellow_cards', 'dispossessed_total', 'red_cards', 
                         'minutes', 'successful_dribbles', 'own_goals', 'interception_total', 
                         'goals_scored', 'key_passes', 'accurateCrosses_total', 'aerialWon_total', 'assists', 
                         'points_above_replacement', 'name', 'age', 'ontarget_scoring_att', 'next_year']

relevant_midfield_keys = ['togga_score', 'apps', 'tackle_total', 'yellow_cards', 'dispossessed_total', 'red_cards', 
                          'minutes', 'successful_dribbles', 'own_goals', 'interception_total', 'ontarget_scoring_att',
                          'goals_scored', 'key_passes', 'accurateCrosses_total', 'clean_sheets', 'aerialWon_total', 
                          'assists', 'points_above_replacement', 'name', 'age', 'next_year']

relevant_defense_keys = ['goals_conceded', 'togga_score', 'apps', 'tackle_total', 'yellow_cards', 'dispossessed_total',
                         'red_cards', 'minutes', 'successful_dribbles', 'own_goals', 'name', 'ontarget_scoring_att',
                         'interception_total', 'goals_scored', 'key_passes', 'accurateCrosses_total', 'clean_sheets', 
                         'aerialWon_total', 'assists', 'points_above_replacement', 'clearance_total', 'age', 'next_year']

relevant_goalie_keys = ['goals_conceded', 'saves', 'togga_score', 'apps', 'tackle_total', 'yellow_cards', 'red_cards', 
                        'minutes', 'own_goals', 'interception_total', 'key_passes', 'clean_sheets', 'aerialWon_total', 
                        'penalties_saved', 'points_above_replacement', 'name', 'clearance_total', 'age', 'next_year']

def removeExtraStats(file, relevant_keys):
    with open(file, "r") as infile:
        data = json.load(infile)
    new_data = []
    for i in range(len(data)):
        values = [{key: data[i].get(key)} for key in relevant_keys]
        item = dict(pair for d in values for pair in d.items())
        new_data.append(item)
    with open(file, 'w') as json_data:
        json.dump(new_data, json_data)
        
for position in ['goalies', 'defense', 'midfield', 'forwards']:
    for year in ['2012/13', '2013/14', '2014/15', '2015/16', '2016/17']:
        file = 'Data/Sorted_Positions/sorted_%s_%s+%s.json' % (position, year[:4], year[-2:])
        if position == 'goalies':
            keys = relevant_goalie_keys
        elif position == 'defense':
            keys = relevant_defense_keys
        elif position == 'midfield':
            keys = relevant_midfield_keys
        else:
            keys = relevant_forward_keys
        removeExtraStats(file, keys)

import random

def sogRandom(file, maxVal):
    with open(file, "r") as infile:
        data = json.load(infile)
    for player in data:
        if not player.get("ontarget_scoring_att"):
            player["ontarget_scoring_att"] = random.randint(0,maxVal)
    with open(file, 'w') as json_data:
        json.dump(data, json_data)

for year in ['2012/13', '2013/14', '2014/15', '2015/16', '2016/17']:
    file = 'Data/Sorted_Positions/sorted_defense_%s+%s.json' % (year[:4], year[-2:])
    sogRandom(file,2)
    file = 'Data/Sorted_Positions/sorted_midfield_%s+%s.json' % (year[:4], year[-2:])
    sogRandom(file,5)
    file = 'Data/Sorted_Positions/sorted_forwards_%s+%s.json' % (year[:4], year[-2:])
    sogRandom(file,7)

In [39]:
with open('Data/Sorted_Positions/sorted_forwards_2016+17.json', "r") as infile:
    data = json.load(infile)
df = pd.DataFrame(data)
df = df.dropna(axis=1,how='all')
df = df.dropna(axis=0,how='any')

#print(df)
#df.shape has 38 entries

with open('Data/Sorted_Positions/sorted_forwards_2015+16.json', "r") as infile:
    data = json.load(infile)
df2 = pd.DataFrame(data)
df2 = df2.dropna(axis=1,how='all')
df2 = df2.dropna(axis=0,how='any')

df = df.set_index("name")
df.index.name = None
df2 = df2.set_index("name")
df2.index.name = None

df3 = df2.add(df)
df3 = df3.dropna(axis=1,how='all')
df3 = df3.dropna(axis=0,how='any')

df3.sort_values('togga_score', 0, False)

#df3.shape
#df3 has 17 entries

# Average age of forwards vs. fantasy points
#age_average = df[['age','togga_score']].groupby('age', as_index=False).mean()
#age_average.plot(x='age', y='togga_score', figsize=(15,10), grid=True)


In [38]:
#print(pd.DataFrame(df, columns = list(['name', 'togga_score', 'points_above_replacement'])))
#print(df[["togga_score", "points_above_replacement"]])
#df
#df[["togga_score", "points_above_replacement"]].describe()
#df[["togga_score", "points_above_replacement"]].corr
#df.any()
#%matplotlib inline

#df.hist(column='togga_score', figsize=(15,10))
#df.boxplot(column='togga_score', figsize=(15,20))

#is_highly_rated = df['togga_score'] >= 300.0
#df[is_highly_rated]


In [211]:
with open('Data/Sorted_Positions/sorted_midfield_2016+17.json', "r") as infile:
    data = json.load(infile)
df = pd.DataFrame(data)
df = df.dropna(axis=1,how='all')
df = df.dropna(axis=0,how='any')

#df has 129 entries

In [213]:
#df

In [215]:
with open('Data/Sorted_Positions/sorted_defense_2016+17.json', "r") as infile:
    data = json.load(infile)
    
df = pd.DataFrame(data)
df = df.dropna(axis=1,how='all')
df = df.dropna(axis=0,how='any')

#df has 86 entries

In [217]:
#df

## Machine Learning

Load all the one year data for forwards

In [115]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

In [317]:
with open("Data/positional.json", "r") as infile:
        data = json.load(infile)
        
forwards = []
for player in data['4']:
    forwards.append(pd.DataFrame([player['history']['2015/16']]))
    forwards.append(pd.DataFrame([player['history']['2014/15']]))
    forwards.append(pd.DataFrame([player['history']['2013/14']]))
    forwards.append(pd.DataFrame([player['history']['2012/13']]))

forwards = pd.concat(forwards)
filter = forwards['togga_score'] > 0
forwards = forwards[filter]
keys = relevant_forward_keys.copy()
keys.remove('points_above_replacement')
forwards = forwards[keys]
forwards = forwards.dropna(axis=1,how='all')
forwards = forwards.dropna(axis=0,how='any')

forwards = forwards.set_index("name")
forwards.index.name = None
keys.remove('name')

#print(forwards.describe())
#forwards.sort_values('togga_score', 0, False).head()



#for year in ['2012/13', '2013/14', '2014/15', '2015/16', '2016/17']:


# Linear Regressor

In [318]:
x_keys = keys.copy()
x_keys.remove('next_year')
X = forwards[x_keys]
Y = forwards['next_year']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=324)
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
Y_prediction = regressor.predict(X_test)
#X_test, y_prediction

RMSE = sqrt(mean_squared_error(y_true = Y_test, y_pred = Y_prediction))
print(RMSE)




100.95872239373855


# Decision Tree Regressor

In [319]:
regressor = DecisionTreeRegressor(max_depth=20)
regressor.fit(X_train, Y_train)
y_prediction = regressor.predict(X_test)
#y_prediction
#X_train.head()

RMSE = sqrt(mean_squared_error(y_true = y_test, y_pred = y_prediction))
print(RMSE)

214.53169281535588


In [137]:
import sys
import scipy
import numpy
import matplotlib
import sklearn

In [140]:
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [148]:
# box and whisker plots of some of the stats
plot_keys = ['togga_score', 'goals_scored', 'assists', 'ontarget_scoring_att', 'aerialWon_total', 'minutes', 
             'successful_dribbles', 'age', 'next_year']
forwards[plot_keys].plot(kind='box', subplots=True, layout=(3,3), sharex=False, sharey=False)
plt.show()

In [147]:
# histograms
forwards[plot_keys].hist()
plt.show()

# Classification ML: Categorization of improvement

In [327]:
forwards["category"] = forwards["togga_score"]
for index, row in forwards.iterrows():
    if row['next_year'] > 0:
        ratio = row['togga_score'] / row['next_year']
        if  ratio < 0.5:
            val = 0
        elif ratio < 0.8:
            val = 1
        elif ratio < 1.0:
            val = 2
        elif ratio < 1.2:
            val = 3
        elif ratio < 1.5:
            val = 4
        else:
            val = 5
        #print(ratio,val)
    else:
        #row[x_keys] = 0
        #forwards.drop(index, inplace=True)
        val = -1
    forwards.set_value(index,'category',val)

    
x_keys = keys.copy()
x_keys.remove('next_year')
x_keys.append("category")
filter = forwards['next_year'] > 0
forwards = forwards[filter]
array = forwards[x_keys].values
X = array[:,:-1]
Y = array[:,-1].astype(int)
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = cross_validation.train_test_split(X, Y, test_size=validation_size, random_state=seed)
#print(X.shape)
#print(X_train.shape)
#print(forwards[x_keys].columns.values)
#print([x_keys])


seed = 7
scoring = 'accuracy'

models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))

# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = cross_validation.KFold(len(X_train), n_folds=10, random_state=seed)
    cv_results = cross_validation.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.490000 (0.185023)
LDA: 0.546667 (0.178388)
KNN: 0.306667 (0.210185)


C:\Users\Noah\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\Noah\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\Noah\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\Noah\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\Noah\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\Noah\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
C:\Users\Noah\Anaconda3\lib\

CART: 0.680000 (0.214580)
NB: 0.546667 (0.178388)
SVM: 0.410000 (0.220126)


In [255]:
# Compare Algorithms
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

# Predictions

In [328]:
# Make predictions on validation dataset
def makePredictions(model):
    model.fit(X_train, Y_train)
    predictions = model.predict(X_validation)
    print(accuracy_score(Y_validation, predictions))
    print(confusion_matrix(Y_validation, predictions))
    print(classification_report(Y_validation, predictions))
    
    new_data = []
    for player in data['4']:
        new_data.append(pd.DataFrame([player['history']['2016/17']]))

    new_data = pd.concat(new_data)
    filter = new_data['togga_score'] > 0
    new_data = new_data[filter]
    new_data = new_data.dropna(axis=1,how='all')
    new_data = new_data.dropna(axis=0,how='any')
    new_data = new_data.set_index("name")
    new_data.index.name = None
    
    new_keys = keys.copy()
    new_keys.remove('next_year')
    array = new_data[new_keys].values
    new_X = array[:,:]
    predictions = model.predict(new_X)
    for name, pred in zip(new_data.index.values, predictions):
        print(name,pred)

# Decision Tree Predictions

In [329]:
makePredictions(DecisionTreeClassifier())

0.6
[[1 0 0 0 0]
 [0 2 0 0 0]
 [0 0 1 0 0]
 [0 1 0 0 1]
 [1 0 2 1 5]]
             precision    recall  f1-score   support

          1       0.50      1.00      0.67         1
          2       0.67      1.00      0.80         2
          3       0.33      1.00      0.50         1
          4       0.00      0.00      0.00         2
          5       0.83      0.56      0.67         9

avg / total       0.64      0.60      0.58        15

Olivier Giroud 5
Callum Wilson 5
Benik Afobe 5
Sam Vokes 4
Andre Gray 5
Ashley Barnes 5
Michy Batshuayi 5
Diego Da Silva Costa 2
Romelu Lukaku 2
Abel Hernández 5
Jamie Vardy 1
Shinji Okazaki 5
Christian Benteke 2
Daniel Sturridge 5
Divock Origi 4
Sergio Agüero 5
Wayne Rooney 5
Marcus Rashford 1
Zlatan Ibrahimovic 3
Cristhian Stuani 5
Shane Long 5
Peter Crouch 3
Fabio Borini 5
Jermain Defoe 3
Harry Kane 1
Vincent Janssen 5
Troy Deeney 5
Salomón Rondón 5
Enner Valencia 5
Álvaro Negredo 5
Fernando Llorente 5
Stefano Okaka 5
Islam Slimani 5


## Logistic Regression Predictions

In [330]:
makePredictions(LogisticRegression())

0.466666666667
[[0 0 0 0 1]
 [1 1 0 0 0]
 [0 0 1 0 0]
 [1 1 0 0 0]
 [3 0 1 0 5]]
             precision    recall  f1-score   support

          1       0.00      0.00      0.00         1
          2       0.50      0.50      0.50         2
          3       0.50      1.00      0.67         1
          4       0.00      0.00      0.00         2
          5       0.83      0.56      0.67         9

avg / total       0.60      0.47      0.51        15



C:\Users\Noah\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1074: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Olivier Giroud 5
Callum Wilson 5
Benik Afobe 3
Sam Vokes 3
Andre Gray 5
Ashley Barnes 5
Michy Batshuayi 5
Diego Da Silva Costa 2
Romelu Lukaku 2
Abel Hernández 3
Jamie Vardy 5
Shinji Okazaki 5
Christian Benteke 4
Daniel Sturridge 5
Divock Origi 2
Sergio Agüero 4
Wayne Rooney 5
Marcus Rashford 2
Zlatan Ibrahimovic 5
Cristhian Stuani 5
Shane Long 3
Peter Crouch 3
Fabio Borini 5
Jermain Defoe 5
Harry Kane 1
Vincent Janssen 1
Troy Deeney 4
Salomón Rondón 2
Enner Valencia 5
Álvaro Negredo 5
Fernando Llorente 5
Stefano Okaka 4
Islam Slimani 5


In [331]:
makePredictions(GaussianNB())

0.733333333333
[[1 0 0 0 0]
 [0 2 0 0 0]
 [0 0 1 0 0]
 [0 0 0 1 1]
 [3 0 0 0 6]]
             precision    recall  f1-score   support

          1       0.25      1.00      0.40         1
          2       1.00      1.00      1.00         2
          3       1.00      1.00      1.00         1
          4       1.00      0.50      0.67         2
          5       0.86      0.67      0.75         9

avg / total       0.86      0.73      0.77        15

Olivier Giroud 5
Callum Wilson 5
Benik Afobe 5
Sam Vokes 5
Andre Gray 5
Ashley Barnes 4
Michy Batshuayi 5
Diego Da Silva Costa 1
Romelu Lukaku 2
Abel Hernández 5
Jamie Vardy 5
Shinji Okazaki 5
Christian Benteke 4
Daniel Sturridge 5
Divock Origi 5
Sergio Agüero 1
Wayne Rooney 1
Marcus Rashford 5
Zlatan Ibrahimovic 1
Cristhian Stuani 5
Shane Long 5
Peter Crouch 3
Fabio Borini 5
Jermain Defoe 5
Harry Kane 1
Vincent Janssen 5
Troy Deeney 1
Salomón Rondón 5
Enner Valencia 5
Álvaro Negredo 5
Fernando Llorente 4
Stefano Okaka 5
Islam Slimani 5
